In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../..")

print('Current location: ', os.getcwd())

Current location:  /home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance


# Visualizations

In [4]:
import os
import pandas as pd

from virny.utils.custom_initializers import create_models_metrics_dct_from_database_df

from source.utils.db_functions import read_model_metric_dfs_from_db
from source.custom_classes.experiments_composer import ExperimentsComposer
from source.custom_classes.experiments_visualizer import ExperimentsVisualizer

## Initialize Configs

In [5]:
EXPERIMENT_NAME = 'mult_repair_levels'
DB_COLLECTION_NAME = f'exp_{EXPERIMENT_NAME}'
DATASET_NAME = 'Folktables_NY_2018_Employment'
EXPERIMENT_SESSION_UUID = '0b71b2a8-030f-44df-a027-5304af77e2ba'
SENSITIVE_ATTRS = ['SEX', 'RAC1P', 'SEX&RAC1P']
MODEL_NAMES = ['RandomForestClassifier']

In [6]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

## Group Metrics Composition

In [7]:
model_metric_dfs = read_model_metric_dfs_from_db(collection_obj, EXPERIMENT_SESSION_UUID)
models_metrics_dct = create_models_metrics_dct_from_database_df(model_metric_dfs)
client.close()

In [8]:
models_metrics_dct[list(models_metrics_dct.keys())[0]].shape

(144, 34)

In [9]:
models_metrics_dct[list(models_metrics_dct.keys())[0]].head(20)

,Metric,Bootstrap_Model_Seed,Model_Name,Model_Params,Run_Number,Dataset_Name,Num_Estimators,Tag,Record_Create_Date_Time,Session_Uuid,...,SEX&RAC1P_priv,SEX&RAC1P_priv_correct,SEX&RAC1P_priv_incorrect,SEX_dis,SEX_dis_correct,SEX_dis_incorrect,SEX_priv,SEX_priv_correct,SEX_priv_incorrect,overall
0,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,100,OK,2023-07-05 14:54:28.064,0b71b2a8-030f-44df-a027-5304af77e2ba,...,0.795506,1.000000,0.000000,0.758308,1.000000,0.000000,0.790467,1.000000,0.000000,0.774500
1,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,100,OK,2023-07-05 15:00:47.485,0b71b2a8-030f-44df-a027-5304af77e2ba,...,0.796629,1.000000,0.000000,0.758308,1.000000,0.000000,0.791460,1.000000,0.000000,0.775000
2,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,100,OK,2023-07-05 15:06:28.925,0b71b2a8-030f-44df-a027-5304af77e2ba,...,0.795506,1.000000,0.000000,0.755287,1.000000,0.000000,0.789474,1.000000,0.000000,0.772500
3,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,100,OK,2023-07-05 15:12:36.130,0b71b2a8-030f-44df-a027-5304af77e2ba,...,0.801124,1.000000,0.000000,0.756294,1.000000,0.000000,0.794439,1.000000,0.000000,0.775500
4,Accuracy,201,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,100,OK,2023-07-05 15:19:26.943,0b71b2a8-030f-44df-a027-5304af77e2ba,...,0.826945,1.000000,0.000000,0.725146,1.000000,0.000000,0.817248,1.000000,0.000000,0.770000
5,Accuracy,201,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,100,OK,2023-07-05 15:25:26.473,0b71b2a8-030f-44df-a027-5304af77e2ba,...,0.821138,1.000000,0.000000,0.734893,1.000000,0.000000,0.816222,1.000000,0.000000,0.774500
6,Accuracy,201,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,100,OK,2023-07-05 15:31:11.575,0b71b2a8-030f-44df-a027-5304af77e2ba,...,0.819977,1.000000,0.000000,0.730994,1.000000,0.000000,0.814168,1.000000,0.000000,0.771500
7,Accuracy,201,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,100,OK,2023-07-05 15:37:35.585,0b71b2a8-030f-44df-a027-5304af77e2ba,...,0.821138,1.000000,0.000000,0.733918,1.000000,0.000000,0.814168,1.000000,0.000000,0.773000
8,Aleatoric_Uncertainty,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,100,OK,2023-07-05 14:54:28.064,0b71b2a8-030f-44df-a027-5304af77e2ba,...,0.526653,0.496423,0.644247,0.564274,0.526943,0.681401,0.540202,0.508265,0.660685,0.552154
9,Aleatoric_Uncertainty,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,100,OK,2023-07-05 15:00:47.485,0b71b2a8-030f-44df-a027-5304af77e2ba,...,0.522707,0.491813,0.643719,0.559975,0.523307,0.675019,0.536881,0.504322,0.660454,0.548347


In [10]:
exp_composer = ExperimentsComposer(models_metrics_dct, SENSITIVE_ATTRS)
exp_subgroup_metrics_dct = exp_composer.create_exp_subgroup_metrics_dct()

In [11]:
exp_group_metrics_dct = exp_composer.compose_group_metrics(exp_subgroup_metrics_dct)

In [12]:
exp_subgroup_metrics_dct['RandomForestClassifier']['Exp_iter_1'][0.2].head()

,Metric,Bootstrap_Model_Seed,Model_Name,Model_Params,Run_Number,Dataset_Name,Num_Estimators,Tag,Record_Create_Date_Time,Session_Uuid,...,SEX&RAC1P_priv,SEX&RAC1P_priv_correct,SEX&RAC1P_priv_incorrect,SEX_dis,SEX_dis_correct,SEX_dis_incorrect,SEX_priv,SEX_priv_correct,SEX_priv_incorrect,overall
1,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,100,OK,2023-07-05 15:00:47.485,0b71b2a8-030f-44df-a027-5304af77e2ba,...,0.796629,1.000000,0.000000,0.758308,1.000000,0.000000,0.791460,1.000000,0.000000,0.775000
9,Aleatoric_Uncertainty,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,100,OK,2023-07-05 15:00:47.485,0b71b2a8-030f-44df-a027-5304af77e2ba,...,0.522707,0.491813,0.643719,0.559975,0.523307,0.675019,0.536881,0.504322,0.660454,0.548347
17,F1,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,100,OK,2023-07-05 15:00:47.485,0b71b2a8-030f-44df-a027-5304af77e2ba,...,0.832562,1.000000,0.000000,0.790941,1.000000,0.000000,0.827303,1.000000,0.000000,0.809645
25,FNR,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,100,OK,2023-07-05 15:00:47.485,0b71b2a8-030f-44df-a027-5304af77e2ba,...,0.136276,0.000000,1.000000,0.190731,0.000000,1.000000,0.144558,0.000000,1.000000,0.167102
33,FPR,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Folktables_NY_2018_Employment,100,OK,2023-07-05 15:00:47.485,0b71b2a8-030f-44df-a027-5304af77e2ba,...,0.298103,0.000000,1.000000,0.307870,0.000000,1.000000,0.298329,0.000000,1.000000,0.303173


In [13]:
exp_group_metrics_dct['RandomForestClassifier']['Exp_iter_1'][0.2].head()

,Metric,SEX,RAC1P,SEX&RAC1P,Model_Name,Experiment_Iteration,Intervention_Param
0,Equalized_Odds_TPR,-0.046173,-0.076982,-0.116597,RandomForestClassifier,Exp_iter_1,0.2
1,Equalized_Odds_FPR,0.009541,-0.039651,-0.064060,RandomForestClassifier,Exp_iter_1,0.2
2,Equalized_Odds_FNR,0.046173,0.076982,0.116597,RandomForestClassifier,Exp_iter_1,0.2
3,Disparate_Impact,0.979700,0.868711,0.812726,RandomForestClassifier,Exp_iter_1,0.2
4,Statistical_Parity_Difference,-0.021681,-0.141317,-0.201293,RandomForestClassifier,Exp_iter_1,0.2


## Metrics Visualization and Reporting

In [14]:
visualizer = ExperimentsVisualizer(exp_subgroup_metrics_dct=exp_subgroup_metrics_dct,
                                   exp_group_metrics_dct=exp_group_metrics_dct,
                                   dataset_name=DATASET_NAME,
                                   model_names=MODEL_NAMES,
                                   sensitive_attrs=SENSITIVE_ATTRS)

In [15]:
visualizer.melted_all_group_metrics_per_model_dct[MODEL_NAMES[0]].head(20)

,Metric,Model_Name,Experiment_Iteration,Intervention_Param,Group,Metric_Value
0,Equalized_Odds_TPR,RandomForestClassifier,Exp_iter_1,0.0,SEX,-0.037506
1,Equalized_Odds_FPR,RandomForestClassifier,Exp_iter_1,0.0,SEX,0.018944
2,Equalized_Odds_FNR,RandomForestClassifier,Exp_iter_1,0.0,SEX,0.037506
3,Disparate_Impact,RandomForestClassifier,Exp_iter_1,0.0,SEX,0.994292
4,Statistical_Parity_Difference,RandomForestClassifier,Exp_iter_1,0.0,SEX,-0.006048
5,Accuracy_Parity,RandomForestClassifier,Exp_iter_1,0.0,SEX,-0.032159
6,Label_Stability_Ratio,RandomForestClassifier,Exp_iter_1,0.0,SEX,0.927994
7,IQR_Parity,RandomForestClassifier,Exp_iter_1,0.0,SEX,0.028735
8,Std_Parity,RandomForestClassifier,Exp_iter_1,0.0,SEX,0.018671
9,Std_Ratio,RandomForestClassifier,Exp_iter_1,0.0,SEX,1.173665


### Line band plots for subgroup metrics

In [16]:
visualizer.create_subgroup_metrics_line_band_plot(model_name=MODEL_NAMES[0],
                                                  subgroup_metrics=['Accuracy', 'F1', 'FNR', 'Jitter', 'Label_Stability', 'FPR'])

alt.VConcatChart(...)

In [17]:
visualizer.create_subgroup_metrics_line_band_plot(model_name=MODEL_NAMES[0],
                                                  subgroup_metrics=['Aleatoric_Uncertainty', 'Overall_Uncertainty'])

alt.VConcatChart(...)

In [18]:
visualizer.create_subgroup_metrics_line_band_plot(model_name=MODEL_NAMES[0],
                                                  subgroup_metrics=['Aleatoric_Uncertainty', 'Overall_Uncertainty'],
                                                  subgroup='SEX_dis_correct')

alt.VConcatChart(...)

In [19]:
visualizer.create_subgroup_metrics_line_band_plot(model_name=MODEL_NAMES[0],
                                                  subgroup_metrics=['Aleatoric_Uncertainty', 'Overall_Uncertainty'],
                                                  subgroup='SEX_dis_incorrect')

alt.VConcatChart(...)

### Line band plots for group metrics

In [20]:
visualizer.create_group_metrics_line_band_plot(group='SEX',
                                               model_name=MODEL_NAMES[0],
                                               group_metrics=['Equalized_Odds_FPR', 'Equalized_Odds_FNR', 'Label_Stability_Ratio', 'Jitter_Parity'])

alt.VConcatChart(...)